## 환경 설정

In [ ]:
!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb
!ls

bareun-linux.deb  drive  sample_data


In [ ]:
!uname -a
!dpkg -i bareun-linux.deb

Linux 316f9e27816a 6.1.85+ #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux
Selecting previously unselected package bareun.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack bareun-linux.deb ...
Unpacking bareun (2.3.0) ...
Setting up bareun (2.3.0) ...
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
Configuration file /etc/systemd/system/bareun.service is marked executable. Please remove executable permission bits. Proceeding anyway.
Created symlink /etc/systemd/system/multi-user.target.wants/bareun.service → /etc/systemd/system/bareun.service.


In [ ]:
!curl -O https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz
!tar -C /opt/bareun -xzf libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  392M  100  392M    0     0  37.3M      0  0:00:10  0:00:10 --:--:-- 40.6M


In [ ]:
%env BAREUN_ROOT="/opt/bareun"
%env LD_LIBRARY_PATH="/opt/bareun/lib"
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun&

env: BAREUN_ROOT="/opt/bareun"
env: LD_LIBRARY_PATH="/opt/bareun/lib"
nohup: appending output to 'nohup.out'


In [ ]:
!ps -ef | grep bareun

root        2046       1 24 06:31 ?        00:00:05 /opt/bareun/bin/bareun
root        2157     410  0 06:31 ?        00:00:00 /bin/bash -c ps -ef | grep bareun
root        2159    2157  0 06:31 ?        00:00:00 grep bareun


In [ ]:
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg 'koba-6AYNELA-VXAUJWQ-SMZ2GFA-HTLCLJI'

/bin/bash: line 1: /opt/bareun/bin/bareun: No such file or directory


In [ ]:
!pip install -U bareunpy

## 테스트 코드 작성

In [ ]:
import sys
import bareunpy as brn
import google.protobuf.text_format as tf

# 아래에 "https://bareun.ai/"에서 이메일 인증 후 발급받은 API KEY("koba-...")를 입력해주세요. "로그인-내정보 확인"
API_KEY = 'API_KEY' # 본인의 API KEY
t = brn.Tagger(API_KEY, "localhost", 5656)
res = t.tags(["안녕하세요. 정말 좋은 날씨네요."])
m = res.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)

sentences {
  text {
    content: "안녕하세요. 정말 좋은 날씨네요."
    length: 18
  }
  tokens {
    text {
      content: "안녕하세요."
      length: 6
    }
    morphemes {
      text {
        content: "안녕하"
        length: 3
      }
      tag: VA
      out_of_vocab: IN_BUILTIN_DICT
    }
    morphemes {
      text {
        content: "시"
        begin_offset: 3
        length: 1
      }
      tag: EP
      probability: 0.9998893
    }
    morphemes {
      text {
        content: "어요"
        begin_offset: 3
        length: 2
      }
      tag: EF
      probability: 0.9899567
    }
    morphemes {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      tag: SF
      probability: 0.99979943
    }
    lemma: "안녕하"
    tagged: "안녕하/VA+시/EP+어요/EF+./SF"
  }
  tokens {
    text {
      content: "정말"
      begin_offset: 7
      length: 2
    }
    morphemes {
      text {
        content: "정말"
        begin_offset: 7
        length: 2
      }
      tag: MAG
      probabili

In [ ]:
res = t.tags(["안녕하세요. 정말 좋은 날씨네요."])
m = res.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)

In [ ]:
res.morphs()

['안녕하', '시', '어요', '.', '정말', '좋', '은', '날씨', '이', '네', '요', '.']

In [ ]:
res = t.tags(["안녕하세요. 정말 좋은 날씨네요."])
res.pos()

[('안녕하', 'VA'),
 ('시', 'EP'),
 ('어요', 'EF'),
 ('.', 'SF'),
 ('정말', 'MAG'),
 ('좋', 'VA'),
 ('은', 'ETM'),
 ('날씨', 'NNG'),
 ('이', 'VCP'),
 ('네', 'EF'),
 ('요', 'JX'),
 ('.', 'SF')]

## 바른 형태소 분석

In [ ]:
import pandas as pd

spelled = pd.read_csv('/content/drive/MyDrive/Final/전처리/불용어_처리/spelled_okt.csv')
spelled.head()

In [ ]:
# 형태소 분석
spelled['bareun'] = spelled['new_content_cleaned_spelled'].apply(lambda x: t.tags([x]).morphs())

In [ ]:
# 형태소 분석 + 품사
spelled['bareun_pos'] = spelled['new_content_cleaned_spelled'].apply(lambda x: t.tags([x]).pos())

In [ ]:
# 저장
spelled.to_csv('/content/drive/MyDrive/Final/전처리/불용어_처리/spelled_okt_bareun.csv', index=False)